In [3]:
!pip install kaggle

In [4]:
!pip uninstall -y numpy


Found existing installation: numpy 2.1.3
Uninstalling numpy-2.1.3:
  Successfully uninstalled numpy-2.1.3


In [5]:
!pip install --force-reinstall matplotlib

  Using cached matplotlib-3.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.54.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (163 kB)
  Using cached kiwisolver-1.4.7-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (6.3 kB)
  Using cached numpy-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pillow-11.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached matplotlib-3.9.2-cp310-

In [6]:

!pip install scikit-learn

ERROR: Operation cancelled by user


In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


**Import Twitter sentiment dataset**


In [1]:
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
# we will extract the compressed file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


**Importing Dependencies**


In [3]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer  # Corrected import path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  # Fixed spelling for sklearn
from sklearn.metrics import accuracy_score


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Data Processing**


In [6]:
#Loading the data from CSV File to Pandas DF
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , encoding = 'ISO-8859-1')

In [7]:
#checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [8]:
#print the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [9]:
#naming the columns and reading th DataSet again
column_names = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , encoding = 'ISO-8859-1' , names = column_names)
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
twitter_data.shape

(1600000, 6)

In [11]:
#counting the missing values
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
#checking the distribute values
twitter_data['target'].value_counts()

**Convert Target "4" to "1"**

In [12]:
twitter_data.replace({'target' :{4:1}}, inplace=True)

In [13]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000





*   **if its (1) means a positive tweet**
*   **if its (0) means a negative tweet**



**Stemming**

In [14]:
port_stem = PorterStemmer()

In [15]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [16]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming )

In [17]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [18]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [19]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [20]:
# we will separate the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [21]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [22]:
print(Y)

[0 0 0 ... 1 1 1]


**Spliting tha data to training and test data**

In [23]:
X_train , X_test , Y_train , Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [24]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(1600000,)
(1280000,)
(320000,)


In [25]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [26]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [27]:
#converting the textual data to numerical data

vectorizer = TfidfVectorizer()


X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.fit_transform(X_test)

In [28]:
print(X_train)

  (np.int32(0), np.int32(436713))	0.27259876264838384
  (np.int32(0), np.int32(354543))	0.3588091611460021
  (np.int32(0), np.int32(185193))	0.5277679060576009
  (np.int32(0), np.int32(109306))	0.3753708587402299
  (np.int32(0), np.int32(235045))	0.41996827700291095
  (np.int32(0), np.int32(443066))	0.4484755317023172
  (np.int32(1), np.int32(160636))	1.0
  (np.int32(2), np.int32(109306))	0.4591176413728317
  (np.int32(2), np.int32(124484))	0.1892155960801415
  (np.int32(2), np.int32(407301))	0.18709338684973031
  (np.int32(2), np.int32(129411))	0.29074192727957143
  (np.int32(2), np.int32(406399))	0.32105459490875526
  (np.int32(2), np.int32(433560))	0.3296595898028565
  (np.int32(2), np.int32(77929))	0.31284080750346344
  (np.int32(2), np.int32(443430))	0.3348599670252845
  (np.int32(2), np.int32(266729))	0.24123230668976975
  (np.int32(2), np.int32(409143))	0.15169282335109835
  (np.int32(2), np.int32(178061))	0.1619010109445149
  (np.int32(2), np.int32(150715))	0.18803850583207948


In [29]:
print(X_test)

  (np.int32(0), np.int32(104606))	0.4459147041194029
  (np.int32(0), np.int32(51603))	0.25774294336856257
  (np.int32(0), np.int32(107797))	0.17983918796121573
  (np.int32(0), np.int32(158685))	0.3174206399662331
  (np.int32(0), np.int32(26316))	0.2701195948137176
  (np.int32(0), np.int32(163496))	0.180932348330391
  (np.int32(0), np.int32(67046))	0.28383380524479984
  (np.int32(0), np.int32(11837))	0.16338171192787918
  (np.int32(0), np.int32(150603))	0.2214371376881977
  (np.int32(0), np.int32(5650))	0.17295837060294847
  (np.int32(0), np.int32(154815))	0.3501587145032445
  (np.int32(0), np.int32(41330))	0.3621075009813407
  (np.int32(0), np.int32(53904))	0.23662705066222536
  (np.int32(1), np.int32(5650))	0.181974929607951
  (np.int32(1), np.int32(2443))	0.2650067441796073
  (np.int32(1), np.int32(99057))	0.2461083000957446
  (np.int32(1), np.int32(141842))	0.21193862113578937
  (np.int32(1), np.int32(134752))	0.4128221547743352
  (np.int32(1), np.int32(83954))	0.35024490258566765
 

**Training the machine learning model**

*Logistique Regression*

In [3]:
model = LogisticRegression(max_iter=1000)


NameError: name 'LogisticRegression' is not defined

In [31]:
model.fit(X_train , Y_train)

LogisticRegression(max_iter=1000)

*Model Evaluation*

Accuracy score


In [32]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [33]:
print('Accuracy Score on the training data : ' , training_data_accuracy)

Accuracy Score on the training data :  0.79871953125


In [2]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score( Y_test ,X_test_prediction)

NameError: name 'model' is not defined

In [35]:
print('Accuracy Score on the training data : ' , test_data_accuracy)

NameError: name 'test_data_accuracy' is not defined

Saving the trained model

In [ ]:
import pickle


In [ ]:
filename = 'trained_model.sav'
pickle.dump(model , open(filename,'wb'))

Using the saved model for the future predictions

In [ ]:
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news are Real')
else:
  print('The news  are fake')